In [21]:
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import sys
sys.path.append("../")

import python.utils as utils
import pandas as pd
import re

from python.constants import PATHS

# Tratamento Pré-análise

In [23]:
raw_data = pd.read_csv(PATHS["input"], encoding = 'utf-8')
raw_data.head()

,id_paciente,sexo,obito,bairro,raca_cor,ocupacao,religiao,luz_eletrica,data_cadastro,escolaridade,...,familia_beneficiaria_auxilio_brasil,crianca_matriculada_creche_pre_escola,altura,peso,pressao_sistolica,pressao_diastolica,n_atendimentos_atencao_primaria,n_atendimentos_hospital,updated_at,tipo
0,cd6daa6e-038d-4952-af29-579e62e07f97,male,0,Parada de Lucas,Branca,Não se aplica,Sem religião,True,2021-06-24 00:00:00.000,Fundamental Incompleto,...,0,0,172.0,52.5,110.0,70.0,8,9,2021-06-01 00:00:00.000,historico
1,ad6cecb2-3a44-49ab-b2f3-6f9ffc2e2ec7,male,0,Cidade Nova,Parda,Assistente Administrativo,Evangélica,1,2021-01-07 00:00:00.000,Médio Incompleto,...,1,0,158.0,76.2,140.0,80.0,0,6,2020-06-25 00:00:00.000,historico
2,54e834e7-e722-4daa-8909-cf917a1247e6,male,0,Santa cruz,Parda,Não se aplica,Católica,1,2021-02-18 00:00:00.000,Fundamental Completo,...,0,0,53.0,82.5,180.0,60.0,5,2,2020-03-02 00:00:00.000,historico
3,c6a71e5e-0933-48d1-9d5a-8f448dc37f71,female,False,Bangu,Branca,Representante Comercial Autônomo,Sem religião,1,2022-03-09 10:40:37,Alfabetizado,...,1,1,164.0,90.6,120.0,70.0,0,27,2021-11-05 11:08:17.477,rotineiro
4,d5262a3c-e5d3-4195-a46b-0acd2533e1d6,male,0,Santíssimo,Branca,Técnico Eletricista,Outra,True,2022-12-13 18:30:45,Médio Completo,...,0,0,154.0,8.5,180.0,80.0,25,0,2020-11-11 00:00:00.000,historico


Analisando o dataset, vemos que, por diversas vezes, temos corrupção de alguns dos caracteres:

In [24]:
def find_broken_substrings(df: pd.DataFrame) -> set[str]:

    broken_substrings = set()
    broken_encoding_pattern = r'\\u00[a-zA-Z0-9][a-zA-Z0-9]'

    # Procurando em cada célula do DataFrame uma substring de encoding quebrado
    df.map(
        lambda cell:
            # Caso haja atualiza a lista de encodings de caracteres quebrados
            broken_substrings.update(
                re.findall(broken_encoding_pattern, cell)
            ) if isinstance(cell, str) else None
    )

    return broken_substrings

In [25]:
find_broken_substrings(raw_data)

{'\\u00d4',
 '\\u00e1',
 '\\u00e2',
 '\\u00e3',
 '\\u00e7',
 '\\u00e9',
 '\\u00ea',
 '\\u00ed',
 '\\u00f3',
 '\\u00f4',
 '\\u00fa'}

Isto aparenta indicar que, apesar de o encoding do arquivo realmente ser em UTF-8, alguns dos valores acabaram não sendo codificados corretamente. Ao analisarmos, vemos que **todos fazem parte do conjunto Latin-1 suplementar** do UTF-8 (especificamente do subconjunto de caracteres com sinais diacríticos, como acentos e cedilha).

É possível que este erro possa ter sido causado por algum tipo de incompatibildade entre o sistema usado para os registros e a colação do banco de dados em que ele foi inserido, ou então causado por diferenças de implementação entre a codificação real na exportação os dados do BD e a plataforma em que ele foi carregado (neste caso, o Google Drive).

Como o encoding estava correto, mas estes caracteres permaneciam não decodificados, tentar decodificar novamente em UTF-8 ou em outro padrão de encoding (como Latin-1). Assim, foi necessário aplicar uma solução mais "primitiva": mapear os códigos aos seus respectivos caracteres, e então substituí-los.

Vale notar que isto claramente não seria uma solução de longo prazo para o consumo de um dataset como este em ambiente de produção. Caso isto ocorresse, idealmente deveria-se investigar o fluxo de dados da fonte (*data source*) até o depósito (*data sink*) para resolver os conflitos de encoding, ou pelo menos adicionar esta etapa ao pré-processamento antes do carregamento destes dados, mas como não é possível no contexto deste desafio, farei desta forma.

In [26]:
data = utils.correct_encoding(raw_data)

In [27]:
find_broken_substrings(data)

set()

Como visto acima, agora não há mais problemas de encoding, e podemos analisar propriamente a distribuição de nossos dados.

# Analisando o dataset

Vamos analisar os tipos de dados:

In [28]:
data.dtypes

id_paciente                               object
sexo                                      object
obito                                     object
bairro                                    object
raca_cor                                  object
ocupacao                                  object
religiao                                  object
luz_eletrica                              object
data_cadastro                             object
escolaridade                              object
nacionalidade                             object
renda_familiar                            object
data_nascimento                           object
em_situacao_de_rua                        object
frequenta_escola                           int64
meios_transporte                          object
doencas_condicoes                         object
identidade_genero                         object
meios_comunicacao                         object
orientacao_sexual                         object
possui_plano_saude  

Percebe-se que temos muito mais dados de natureza categórica que numérica. Estes primeiros terão que ser explorados mais a fundo, pois alguns destes podem ser estruturados diferentemente de outros.

No entanto, antes disto, vamos apurar o "perfil" dos dados numéricos.

## Explorando dados numéricos

In [29]:
num_data = data.select_dtypes(['int64', 'float64'])

In [30]:
num_data.describe()

,frequenta_escola,altura,peso,pressao_sistolica,pressao_diastolica,n_atendimentos_atencao_primaria,n_atendimentos_hospital
count,100000.000000,99975.000000,99816.000000,99960.000000,99983.000000,100000.000000,100000.000000
mean,0.158190,142.693589,63.801629,129.544068,79.378207,5.542710,7.312930
std,0.364921,38.659345,35.594173,21.406429,21.062140,5.204464,9.801987
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,145.000000,50.700000,120.000000,70.000000,0.000000,0.000000
50%,0.000000,158.000000,67.900000,130.000000,80.000000,5.000000,2.000000
75%,0.000000,165.000000,82.000000,140.000000,87.000000,9.000000,13.000000
max,1.000000,810.000000,998.000000,900.000000,921.000000,32.000000,77.000000


### Validez dos dados

Esta visão dos dados já nos dá alguma informação. Percebemos que a maioria dos registros numéricos tem algum tipo de preenchimento, o que é bom à primeira vista, pois pode indicar que pelo menos estes dados possam ser utilizados para algum tipo de análise. Resta saber quantos dos valores preenchidos são realmente válidos.

Para este último ponto, vou redirecionar a atenção [à documentação disponibilizada pelo desafio](https://docs.google.com/spreadsheets/d/1xZKK1JJmZzWPONzpPuwAnhOYW2duwFkf/edit?gid=1923416656#gid=1923416656). Na descrição do campo *_altura_*, está registrado apenas "Altura medida". Pelos valores dos quartis do campo (25%, 50% e 75%, respectivamente), podemos inferir que está medido em centímetros, tais que o primeiro quartil seria 145 cm, o segundo 158 cm e o terceiro 165 cm. No entanto, dado este padrão, que lógica há no fato de que o valor **_máximo_** para o campo é 810?

Pode até ser de que a medida tenha sido acidentalmente registrada segundo o sistema imperial de medidas (dos EUA) devido ao paciente ter sido um estrangeiro, e sua altura real ser 8'10" (2,69 m), mas isto é altamente improvável dado que o récorde histórico de altura humana é de 8'11". Igualmente, nenhum ser humano pode pesar 900 Kg, e um coração humano é incapaz de suportar um máximo de pressão de 900 mmHg de sístole, muito menos 921 de diástole. Assim, fica claro de que alguns registros claramente são _outliers_ estatísticos devido a erros de alguma natureza (provavelmente durante o registro).

Geralmente, a forma de lidar com estes casos é manter estes registros e removê-los da consideração de um projeto através de análise estatística, mas em última instância é uma decisão a nível de arquitetura entender se eles deveriam ser mantidos ou removidos. Ademais, deveria ser associado aos registros algum tipo de documentação ou preferencialmente um metadado descrevendo as unidades de medida para cada dado numérico — percebe-se, claramente, que alguns dos campos são medições em uma escala numérica, enquanto outros, como `frequenta_escola`, são valores booleanos (verdadeiro ou falso) codificados como 0 ou 1.

Neste caso, optarei por manter um conjunto de dados maior, e deixarei estas informações conforme estão.

## Explorando dados categóricos

### Padrões diferentes para mesmo tipo de dado

Ainda no tópico de valores booleanos, é curioso notar que, enquanto `frequenta_escola` é registrado numericamente, os seguintes campos, que também aparentam ser booleanos pelas suas descrições, são categóricos:

- `obito`
- `luz_eletrica`
- `em_situacao_de_rua`
- `possui_plano_saude`
- `familia_beneficiaria_auxilio_brasil`
- `crianca_matriculada_creche_pre_escola`

Em todos estes, assim como em `frequenta_escola`, suas descrições começam como "Indicação _se_ [...]", de onde podemos inferir que é um indicador sim/não. Vamos olhar todos estes campos ao mesmo tempo, comparando-os entre si.

In [31]:
data[[
    "obito",
    "luz_eletrica", 
    "em_situacao_de_rua", 
    "frequenta_escola", 
    "possui_plano_saude", 
    "familia_beneficiaria_auxilio_brasil", 
    "crianca_matriculada_creche_pre_escola"
]].head()

,obito,luz_eletrica,em_situacao_de_rua,frequenta_escola,possui_plano_saude,familia_beneficiaria_auxilio_brasil,crianca_matriculada_creche_pre_escola
0,0,True,0,0,False,0,0
1,0,1,0,0,1,1,0
2,0,1,0,0,0,0,0
3,False,1,0,0,0,1,1
4,0,True,0,1,0,0,0


Percebe-se que estes dados, todos de mesma natureza, não possuem padrão nenhum. Seria uma inferência relativamente segura supor que 1 ou qualquer variação de "true" ou "verdadeiro" equivaleria a um valor que indicasse verdade lógica, e que 0 ou qualquer variação de "false" ou "falso" indicasse falsidade lógica.

Assim vendo, poderíamos fazer uma etapa de conformidade, garantindo que todos estes campos sigam a mesma estrutura lógica. Contudo, é importante notar que isto deveria ser verificado tanto na documentação de cada possível fonte destes dados, quanto em possíveis versões anteriores do pipeline de dados já existente (caso já tenha sido criada anteriormente uma etapa de conformidade com regras diferentes).

Neste caso, como não sabemos qual será o SGBD utilizado para guardar estes dados, vou convertê-los em 0 e 1, pois nem todo SGBD tem um tipo `BOOLEAN` (ou algo do gênero), mas como o tipo `BIT` é definido no ANSI-SQL, praticamente todos o implementam, sendo uma solução relativamente agnóstica.

In [32]:
data = data.apply(utils.conform_booleans)

In [33]:
bool_columns = [
    "obito",
    "luz_eletrica", 
    "em_situacao_de_rua", 
    "frequenta_escola", 
    "possui_plano_saude", 
    "familia_beneficiaria_auxilio_brasil", 
    "crianca_matriculada_creche_pre_escola"
]

for col in bool_columns:
    print(data[col].unique())

[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


Vale lembrar que, apesar de estes dados serem _representados_ por números, eles são dados **categóricos**, pois sua forma original não é um valor, mas um sim ou um não.

## Dados multivalorados